Here we will implement an expression class for representing Real valued functions that take vectors as inputs.

In [37]:
import numpy as np

# Node Classes

# Base Expression
# the _b at the end means it supports backward mode
class Expression:
    def __init__(self):
        self.partial = 0
        self.value = 0

    def __str__():
        return "EXPRESSION" 

    def eval(self):
        self.partial = 0
        return "CANT EVALUATE BASE EXPRESSION CLASS"

    def derive():
        return "CANT DERIVE OF BASE EXPRESSION CLASS"

# Base Expression Types
    
class Un_Op(Expression):
    def __init__(self, a):
        Expression.__init__(self)
        self.a = a
    def __str__(self):
        return "[UNARY OP]"

class Bi_Op(Expression):
    def __init__(self, a, b):
        Expression.__init__(self)
        self.a = a
        self.b = b
    def __str__(self):
        return "[BINARY OP]"
    
# Usable Expression Types

# Constants
class Const(Un_Op):
    def __str__(self):
        return str(self.a)
    
    def eval(self):
        self.value = self.a
        return self.a
    def derive(self):
        return 0
    
class VConst(Un_Op):
    def __str__(self):
        return str(self.a)
    
    def eval(self):
        self.value = self.a
        return self.a
    def derive(self):
        return np.zeros(self.a.shape)

# Variables  
class Var(Expression):
    def __init__(self, name, value = None):
        '''
        name  : typeof(string)
        value : typeof(float)
        '''
        self.name = name
        self.value = value
        self.partial = 0
    def __str__(self):
        return self.name
    def eval(self):
        return self.value
    def derive(self):
        return 1
    
class VVar(Expression):
    def __init__(self, name, value = None):
        '''
        name  : typeof(string)
        value : typeof(float)
        '''
        self.name = name
        self.value = value
        self.partial = 0
    def __str__(self):
        return self.name
    def eval(self):
        return self.value
    def derive(self):
        return np.ones(self.a.shape)
    
# Unary Operations
class Sum(Un_Op):
    def __str__(self):
        return f"Sum({str(self.a)})"
    def eval(self):
        self.value = np.sum(self.a.eval())
        return self.value
    def derive(self):
        dthis_da = np.ones(self.a.value.shape)
        self.a.partial += dthis_da * self.partial
        self.a.derive()
        return
    
class Sin(Un_Op):
    def __str__(self):
        return f"sin({str(self.a)})"
    def eval(self):
        self.value = np.sin(self.a.eval())
        return self.value
    def derive(self):
        dthis_da = np.cos(self.a.value)
        self.a.partial += dthis_da * self.partial # check if this is elementwise for np arrays
        self.a.derive()
        return

class Exp(Un_Op):
    def __str__(self):
        return f"exp({str(self.a)})"
    def eval(self):
        self.value = np.exp(self.a.eval())
        return self.value
    def derive(self):
        dthis_da = np.exp(self.a.value)
        self.a.partial += dthis_da * self.partial # check if this is elementwise for np arrays
        self.a.derive()
        return

# Binary Operations
class SAdd(Bi_Op):
    def __str__(self):
        return f"({str(self.a)} + {str(self.b)})"
    def eval(self):
        self.value = self.a.eval() + self.b.eval()
        return self.value    
    def derive(self):
        dthis_da = 1
        dthis_db = 1
        self.a.partial += dthis_da * self.partial
        self.b.partial += dthis_db * self.partial
        self.a.derive()
        self.b.derive()
        return  

class VAdd(Bi_Op):
    def __str__(self):
        return f"({str(self.a)} + {str(self.b)})"
    def eval(self):
        self.value = self.a.eval() + self.b.eval()
        return self.value   
    def derive(self):
        self.a.partial += self.partial # check if this is elementwise for np arrays
        self.b.partial += self.partial # check if this is elementwise for np arrays
        self.a.derive()
        self.b.derive()   
    
class MVMult(Bi_Op):
    def __str__(self):
        return f"({str(self.a)} * {str(self.b)})"  
    def eval(self):
        self.value = np.matmul(self.a.eval(), self.b.eval().squeeze())
        return self.value
    def derive(self): #I think this works? but it needs more testing
        print("self.partial", self.partial)
        dthis_da = np.transpose(self.b.value)
        dthis_db = np.transpose(self.a.value)
        self.a.partial += (dthis_da @ np.array([self.partial])).T # not sure why this works, but it does seem to...
        self.b.partial += dthis_db @ self.partial
        self.a.derive()
        self.b.derive()   

class MMMult(Bi_Op):
    def __str__(self):
        return f"({str(self.a)} * {str(self.b)})"  
    def eval(self):
        self.value = np.matmul(self.a.eval().squeeze(), self.b.eval().squeeze())
        return self.value
    def derive(self): # this doesnt work quite right yet
        print("self.partial", self.partial)
        dthis_da = np.transpose(self.b.value)
        dthis_db = np.transpose(self.a.value)
        self.a.partial += dthis_da @ self.partial
        self.b.partial += self.partial @ dthis_db
        self.a.derive()
        self.b.derive()   

In [39]:
# code that we want to work
A = Var("A", np.array([[2 ,2],[1,0]]))
x = Var("x", np.array([[1,0.5]]))
y = Var("y", np.array([1,0.5])) # note this is disgusting syntax, the fact we have to define vectors differently for addition and matrix multiplication
func = Sum(VAdd(MVMult(A, x), y))
        
print("f =", func.eval())

func.partial = 1

func.derive()

print("df/dA =\n", A.partial)
print("df/dx =\n", x.partial)
print("df/dy =\n", y.partial)

f = 5.5
self.partial [1. 1.]
df/dA =
 [[1.  0.5]
 [1.  0.5]]
df/dx =
 [3. 2.]
df/dy =
 [1. 1.]
